In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [10]:
df = pd.read_csv('/workspaces/Futures-First/BackTest/data/LE Outrights/LE Apr25_1hour.csv')
tick_size = 0.025
tick_price = 10
df['Date'] = pd.to_datetime(df['Date'])
df = df.sort_values(by='Date')
df.set_index('Date', inplace=True)
df

,Open,High,Low,Close
Date,,,,
2024-06-03 13:00:00,189.550,189.650,189.275,189.350
2024-06-03 14:00:00,189.375,189.450,188.625,188.650
2024-06-03 15:00:00,188.650,188.850,188.400,188.525
2024-06-03 16:00:00,188.450,189.500,187.825,189.300
2024-06-03 17:00:00,189.375,189.800,188.925,189.500
...,...,...,...,...
2024-09-17 14:00:00,181.825,182.250,181.450,182.100
2024-09-17 15:00:00,182.125,182.500,181.925,182.050
2024-09-17 16:00:00,182.000,182.425,181.950,182.300


In [11]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Set the tick size and tick price
tick_size = 0.025
tick_price = 10
df = df.reset_index(drop=True)
df.index.name = 'Serial'

# Assuming df is already loaded and preprocessed as in your original code

def calculate_bollinger_bands(df, window=20, std_dev=1.5):
    df['SMA'] = df['Close'].rolling(window=window).mean()
    df['STD'] = df['Close'].rolling(window=window).std()
    df['Upper Band'] = df['SMA'] + (df['STD'] * std_dev)
    df['Lower Band'] = df['SMA'] - (df['STD'] * std_dev)
    return df

def detect_outside_bands(df):
    df['Outside Upper'] = (df['Low'] > df['Upper Band']) & (df['High'] > df['Upper Band'])
    df['Outside Lower'] = (df['Low'] < df['Lower Band']) & (df['High'] < df['Lower Band'])
    return df

def implement_trading_strategy(df):
    df['Position'] = 0
    df['Entry Price'] = np.nan
    df['Exit Price'] = np.nan
    df['Stop Loss'] = np.nan
    df['Exit Type'] = ''
    
    in_position = False
    position_type = None
    entry_index = None
    stop_loss = None

    for i in range(1, len(df)):
        if not in_position:
            if df['Outside Upper'].iloc[i-1]:
                reference_low = df['Low'].iloc[i-1]
                if df['Low'].iloc[i] < reference_low:
                    df.loc[df.index[i], 'Position'] = -1  # Short
                    df.loc[df.index[i], 'Entry Price'] = reference_low
                    stop_loss = df['High'].iloc[i-1]  # Set SL at reference candle high
                    df.loc[df.index[i], 'Stop Loss'] = stop_loss
                    in_position = True
                    position_type = 'short'
                    entry_index = i
            elif df['Outside Lower'].iloc[i-1]:
                reference_high = df['High'].iloc[i-1]
                if df['High'].iloc[i] > reference_high:
                    df.loc[df.index[i], 'Position'] = 1  # Long
                    df.loc[df.index[i], 'Entry Price'] = reference_high
                    stop_loss = df['Low'].iloc[i-1]  # Set SL at reference candle low
                    df.loc[df.index[i], 'Stop Loss'] = stop_loss
                    in_position = True
                    position_type = 'long'
                    entry_index = i
        else:
            if position_type == 'short':
                if df['High'].iloc[i] >= stop_loss:
                    df.loc[df.index[i], 'Position'] = 0
                    df.loc[df.index[i], 'Exit Price'] = stop_loss
                    df.loc[df.index[i], 'Exit Type'] = 'Stop Loss'
                    in_position = False
                    position_type = None
                elif df['High'].iloc[i] > df['High'].iloc[i-1]:
                    df.loc[df.index[i], 'Position'] = 0
                    df.loc[df.index[i], 'Exit Price'] = df['High'].iloc[i-1]
                    df.loc[df.index[i], 'Exit Type'] = 'Trend Break'
                    in_position = False
                    position_type = None
            elif position_type == 'long':
                if df['Low'].iloc[i] <= stop_loss:
                    df.loc[df.index[i], 'Position'] = 0
                    df.loc[df.index[i], 'Exit Price'] = stop_loss
                    df.loc[df.index[i], 'Exit Type'] = 'Stop Loss'
                    in_position = False
                    position_type = None
                elif df['Low'].iloc[i] < df['Low'].iloc[i-1]:
                    df.loc[df.index[i], 'Position'] = 0
                    df.loc[df.index[i], 'Exit Price'] = df['Low'].iloc[i-1]
                    df.loc[df.index[i], 'Exit Type'] = 'Trend Break'
                    in_position = False
                    position_type = None

    return df

def calculate_pnl(df, tick_size, tick_price):
    df['PnL'] = np.nan
    entry_price = None
    position = 0
    
    for i in range(len(df)):
        if pd.notnull(df['Entry Price'].iloc[i]):
            entry_price = df['Entry Price'].iloc[i]
            position = df['Position'].iloc[i]
        elif pd.notnull(df['Exit Price'].iloc[i]) and entry_price is not None:
            exit_price = df['Exit Price'].iloc[i]
            pnl = (exit_price - entry_price) * position
            df.loc[df.index[i], 'PnL'] = pnl / tick_size * tick_price
            entry_price = None
            position = 0
    
    return df

def print_trade_details(df):
    entries = df[df['Position'] != 0].copy()
    exits = df[df['Exit Price'].notnull()].copy()
    
    trades = []
    for i, entry in entries.iterrows():
        exit_row = exits.loc[exits.index > i].iloc[0] if not exits.loc[exits.index > i].empty else None
        if exit_row is not None:
            trade = {
                'Entry Serial': i,
                'Entry Price': entry['Entry Price'],
                'Position': 'Long' if entry['Position'] == 1 else 'Short',
                'Exit Serial': exit_row.name,
                'Exit Price': exit_row['Exit Price'],
                'PnL': exit_row['PnL']
            }
            trades.append(trade)

    print("\nIndividual Trade Details:")
    print("-------------------------")
    for i, trade in enumerate(trades, 1):
        print(f"Trade {i}:")
        print(f"Entry Serial: {trade['Entry Serial']}")
        print(f"Entry Price: {trade['Entry Price']:.2f}")
        print(f"Position: {trade['Position']}")
        print(f"Exit Serial: {trade['Exit Serial']}")
        print(f"Exit Price: {trade['Exit Price']:.2f}")
        print(f"PnL: ${trade['PnL']:.2f}")
        print("-------------------------")

    pnl_series = pd.Series([trade['PnL'] for trade in trades])
    print(f"\nTotal PnL: ${pnl_series.sum():.2f}")
    print(f"Number of trades: {len(trades)}")
    print(f"Win rate: {(pnl_series > 0).mean():.2%}")
    print(f"Average PnL per trade: ${pnl_series.mean():.2f}")


    
def plot_strategy(df):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, 
                        vertical_spacing=0.03, subplot_titles=('Price and Bollinger Bands', 'PnL'),
                        row_heights=[0.7, 0.3])

    # Candlestick chart
    fig.add_trace(go.Candlestick(x=df.index,
                                 open=df['Open'],
                                 high=df['High'],
                                 low=df['Low'],
                                 close=df['Close'],
                                 name='Price'),
                  row=1, col=1)

    # Bollinger Bands
    fig.add_trace(go.Scatter(x=df.index, y=df['Upper Band'], name='Upper Band', line=dict(color='red', width=1)), row=1, col=1)
    fig.add_trace(go.Scatter(x=df.index, y=df['Lower Band'], name='Lower Band', line=dict(color='green', width=1)), row=1, col=1)

    # Entry and Exit points
    long_entries = df[df['Position'] == 1]
    short_entries = df[df['Position'] == -1]
    exits = df[df['Exit Price'].notnull()]
    stop_losses = df[df['Stop Loss'].notnull()]

    fig.add_trace(go.Scatter(x=long_entries.index, y=long_entries['Entry Price'], mode='markers', 
                             name='Long Entry', marker=dict(color='green', size=10, symbol='triangle-up')), row=1, col=1)
    fig.add_trace(go.Scatter(x=short_entries.index, y=short_entries['Entry Price'], mode='markers', 
                             name='Short Entry', marker=dict(color='red', size=10, symbol='triangle-down')), row=1, col=1)
    fig.add_trace(go.Scatter(x=exits.index, y=exits['Exit Price'], mode='markers', 
                             name='Exit', marker=dict(color='black', size=8, symbol='x')), row=1, col=1)
    fig.add_trace(go.Scatter(x=stop_losses.index, y=stop_losses['Stop Loss'], mode='markers',
                             name='Stop Loss', marker=dict(color='purple', size=8, symbol='square-open')), row=1, col=1)

    # PnL
    fig.add_trace(go.Bar(x=df.index, y=df['PnL'], name='PnL'), row=2, col=1)

    fig.update_layout(title='Bollinger Band Trading Strategy with Stop-Loss', height=800, width=1200,
                      showlegend=True, xaxis_rangeslider_visible=False)
    
    # Update x-axis and y-axis labels
    fig.update_xaxes(title_text="Serial Number", row=2, col=1)
    fig.update_yaxes(title_text="Price", row=1, col=1)
    fig.update_yaxes(title_text="PnL", row=2, col=1)

    fig.show()

# Apply the strategy
df = calculate_bollinger_bands(df)
df = detect_outside_bands(df)
df = implement_trading_strategy(df)
df = calculate_pnl(df, tick_size, tick_price)

# Print trade details
print_trade_details(df)

# Plot the strategy
plot_strategy(df)


Individual Trade Details:
-------------------------
Trade 1:
Entry Serial: 22
Entry Price: 187.75
Position: Long
Exit Serial: 24
Exit Price: 187.25
PnL: $-200.00
-------------------------
Trade 2:
Entry Serial: 34
Entry Price: 189.85
Position: Short
Exit Serial: 35
Exit Price: 190.20
PnL: $-140.00
-------------------------
Trade 3:
Entry Serial: 59
Entry Price: 191.82
Position: Short
Exit Serial: 60
Exit Price: 192.25
PnL: $-170.00
-------------------------
Trade 4:
Entry Serial: 80
Entry Price: 190.88
Position: Long
Exit Serial: 82
Exit Price: 190.57
PnL: $-120.00
-------------------------
Trade 5:
Entry Serial: 85
Entry Price: 190.28
Position: Long
Exit Serial: 86
Exit Price: 189.90
PnL: $-150.00
-------------------------
Trade 6:
Entry Serial: 117
Entry Price: 188.93
Position: Long
Exit Serial: 118
Exit Price: 188.62
PnL: $-120.00
-------------------------
Trade 7:
Entry Serial: 168
Entry Price: 189.90
Position: Short
Exit Serial: 170
Exit Price: 189.75
PnL: $60.00
----------------